In [1]:
from neuralintents import GenericAssistant
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
from pandas_datareader import data as pdr
import mplfinance as mpf
import speech_recognition
import pyttsx3 as tts
import sys
import pickle
import datetime as dt
engine = tts.init()
voices = engine.getProperty('voices')
for voice in voices:
    print("Voice: %s" % voice.name)
    print(" - ID: %s" % voice.id)
    print(" - Languages: %s" % voice.languages)
    print(" - Gender: %s" % voice.gender)
    print(" - Age: %s" % voice.age)
    print("\n")

engine.setProperty("voice", voices[2].id)

Voice: Microsoft David Desktop - English (United States)
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
 - Languages: []
 - Gender: None
 - Age: None


Voice: Microsoft Zira Desktop - English (United States)
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
 - Languages: []
 - Gender: None
 - Age: None


Voice: Microsoft Sabina Desktop - Spanish (Mexico)
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ES-MX_SABINA_11.0
 - Languages: []
 - Gender: None
 - Age: None


Voice: Microsoft Heami Desktop - Korean
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_KO-KR_HEAMI_11.0
 - Languages: []
 - Gender: None
 - Age: None




In [2]:
recognizer=speech_recognition.Recognizer()
speaker=tts.init()
speaker.setProperty('rate',150)

In [3]:
portafolio={'facebook':40,"apple":33,"amazon":15,"cisco":24,"coca cola":12,"intel":31}

with open('portafolio.pkl','wb') as f:
    pickle.dump(portafolio,f)
    
with open('portafolio.pkl','rb') as f:
    portafolio = pickle.load(f)

In [4]:
def hola():
    speaker.say("Hola, en que puedo ayudarte?")
    speaker.runAndWait()

In [5]:
def salir():
    speaker.say("Espero haberte ayudado")
    speaker.runAndWait()
    sys.exit(0)

In [6]:
def grafica_accion():
    global recognizer
    speaker.say("Escriba el indicador de la cotizacion que desea ver?")
    speaker.runAndWait()    
    ticker = input("Elija el simbolo de cotizacion: ")
    
    plt.style.use('dark_background')
    
    start = dt.datetime(2020,1,1)
    end = dt.datetime.now()
    
    data=pdr.get_data_yahoo(ticker,start=start, end=end)
    
    colors = mpf.make_marketcolors(up='#00ff00',down='#ff0000',wick='inherit',edge='inherit',volume='in')
    mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=colors)
    mpf.plot(data, type='candle',style=mpf_style,volume=True)
               
    speaker.say(f"La grafica de la cotizacion es la siguiente ")
    speaker.runAndWait() 
    
    sum=0
    data=pdr.get_data_yahoo(ticker)
    price = data['Close'].iloc[-1]-data['Close'].iloc[-2]
    sum += price
    if sum<0:
        speaker.say(f"la accion disminuyo {round(abs(sum),2)} dolares de ayer a hoy")
        speaker.runAndWait() 
        done = True
    if sum>0:
        speaker.say(f"la accion aumento {round(abs(sum),2)} dolares de ayer a hoy")
        speaker.runAndWait() 
        done = True
    if sum==0:
        speaker.say(f"la accion ha permanecido estable en {round(abs(sum),2)} dolares de ayer a hoy")
        speaker.runAndWait() 
        done = True

In [22]:
def lista():
    global recognizer
    speaker.say("Cual empresa desea agregar?")
    speaker.runAndWait()    
    done = False
    while not done:
        try:
            with speech_recognition.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic,duration=0.2)
                audio=recognizer.listen(mic)
                ticker=recognizer.recognize_google(audio,language="es-CO")#item
                ticker=ticker.lower()#item
 
                speaker.say(f"Cuantas acciones de {ticker} desea agregar?")
                speaker.runAndWait()
                recognizer.adjust_for_ambient_noise(mic,duration=0.2)
                audio=recognizer.listen(mic)
                amount=recognizer.recognize_google(audio,language="es-CO")#item
                amount=amount.lower()#item
                speaker.say(f"Sean agregado {amount} acciones de {ticker}") 
                if ticker in portafolio.keys():
                    portafolio[ticker] += int(amount)
                else:
                    portafolio[ticker] + int(amount)
                speaker.say(f"Tienes {portafolio[ticker]} acciones de {ticker} en el portafolio")
                speaker.runAndWait()
                done = True
                
        except speech_recognition.UnknownValueError:
            recognizer=speech_recognition.Recognizer()
            speaker.say("No entendi lo que dijiste, por favor podrias repetir?")
            speaker.runAndWait()
            
with open('portafolio.pkl','wb') as f:
        pickle.dump(portafolio,f)

In [ ]:
def vender():
    global recognizer
    speaker.say("Cual accion desea vender?")
    speaker.runAndWait()    
    done = False
    while not done:
        try:
            with speech_recognition.Microphone() as mic:
                recognizer.adjust_for_ambient_noise(mic,duration=0.2)
                audio=recognizer.listen(mic)
                ticker=recognizer.recognize_google(audio,language="es-CO")#item
                ticker=ticker.lower()#item
                
                speaker.say(f"Cuantas acciones de {ticker} desea vender?")
                speaker.runAndWait()
                recognizer.adjust_for_ambient_noise(mic,duration=0.2)
                audio=recognizer.listen(mic)
                amount=recognizer.recognize_google(audio,language="es-CO")#item
                amount=amount.lower()#item
                
                if ticker in portafolio.keys():
                    if int(amount) <= portafolio[ticker]:
                        speaker.say(f"Sean vendido {amount} acciones de {ticker}") 
                        portafolio[ticker] -= int(amount)
                        speaker.say(f"Te quedan {portafolio[ticker]} acciones de {ticker} en el portafolio")
                        speaker.runAndWait()
                        done = True
                    else:
                        speaker.say(f"No tiene suficientes acciones de {ticker} para vender")
                        speaker.runAndWait()
                        done = True
                else:
                    speaker.say(f"No tiene acciones de {ticker}")
                    speaker.runAndWait()
                    done = True
                    
        except speech_recognition.UnknownValueError:
            recognizer=speech_recognition.Recognizer()
            speaker.say("No entendi lo que dijiste, por favor podrias repetir?")
            speaker.runAndWait()
            
with open('portafolio.pkl','wb') as f:
        pickle.dump(portafolio,f)


In [24]:
def presentar_lista():
    speaker.say("Las acciones en el portafolio son:")
    for ticker in portafolio.keys():
        speaker.say(f"{portafolio[ticker]} acciones de {ticker} ")
        speaker.runAndWait()

In [25]:
mappings={
    "inicio":hola,
    "salir": salir,
    "lista": lista,
    "presentar": presentar_lista,
    "vender": vender,
    "grafica_accion":grafica_accion,
}

In [26]:
assistant=GenericAssistant('palabras.json',intent_methods=mappings)
assistant.train_model()

while True:
    try:
        with speech_recognition.Microphone() as mic:
            recognizer.adjust_for_ambient_noise(mic,duration=0.2)
            audio=recognizer.listen(mic)
            message=recognizer.recognize_google(audio,language="es-CO")
            message=message.lower()
        assistant.request(message)
    except speech_recognition.UnknownValueError:
        recognizer=speech_recognition.Recognizer()

Epoch 1/200
5/5 [==============================] - 0s 1ms/step - loss: 1.8884 - accuracy: 0.0800
Epoch 2/200
5/5 [==============================] - 0s 1ms/step - loss: 1.7462 - accuracy: 0.1600
Epoch 3/200
5/5 [==============================] - 0s 1ms/step - loss: 1.6292 - accuracy: 0.4400
Epoch 4/200
5/5 [==============================] - 0s 1ms/step - loss: 1.4307 - accuracy: 0.6000
Epoch 5/200
5/5 [==============================] - 0s 2ms/step - loss: 1.5086 - accuracy: 0.4400
Epoch 6/200
5/5 [==============================] - 0s 2ms/step - loss: 1.4457 - accuracy: 0.4400
Epoch 7/200
5/5 [==============================] - 0s 2ms/step - loss: 1.3267 - accuracy: 0.4400
Epoch 8/200
5/5 [==============================] - 0s 1ms/step - loss: 1.1690 - accuracy: 0.5200
Epoch 9/200
5/5 [==============================] - 0s 1ms/step - loss: 1.0975 - accuracy: 0.5200
Epoch 10/200
5/5 [==============================] - 0s 2ms/step - loss: 0.9214 - accuracy: 0.7600
Epoch 11/200
5/5 [===========

5/5 [==============================] - 0s 1ms/step - loss: 0.0160 - accuracy: 1.0000
Epoch 85/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0358 - accuracy: 1.0000
Epoch 86/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0631 - accuracy: 1.0000
Epoch 87/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0883 - accuracy: 1.0000
Epoch 88/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0734 - accuracy: 1.0000
Epoch 89/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0988 - accuracy: 0.9600
Epoch 90/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0665 - accuracy: 1.0000
Epoch 91/200
5/5 [==============================] - 0s 1ms/step - loss: 0.1055 - accuracy: 1.0000
Epoch 92/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0600 - accuracy: 0.9600
Epoch 93/200
5/5 [==============================] - 0s 1ms/step - loss: 0.1083 - accuracy: 0.9600
Epoch 94/200
5/5 [===============

5/5 [==============================] - 0s 1ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 168/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0108 - accuracy: 1.0000
Epoch 169/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 170/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0302 - accuracy: 1.0000
Epoch 171/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0197 - accuracy: 1.0000
Epoch 172/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0949 - accuracy: 0.9600
Epoch 173/200
5/5 [==============================] - 0s 1ms/step - loss: 0.1119 - accuracy: 0.9600
Epoch 174/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0223 - accuracy: 1.0000
Epoch 175/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0145 - accuracy: 1.0000
Epoch 176/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 177/200
5/5 [=====

KeyboardInterrupt: 